/Users/mahaabu-khousa/neuefische/NeuralXpresso/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [61]:
import neuralxpresso as nx
import plots as plots
from plotly.subplots import make_subplots
YT_LINK = 'https://www.youtube.com/watch?v=2ka4Rpq9KCw&ab_channel=Blinderskin'
nx_session = nx.NeuralXpressoSession(yt_link=YT_LINK)
result = nx_session.run_analysis(video_output=True)




1/1 [==============================] - 0s 304ms/step


2023-03-29 18:34:18.471662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 14ms/step


In [41]:
result['new_export']['main_character_data'][1]

,frame,emotion,probability
0,1,Angry,0.047856
1,2,Angry,0.062015
2,3,Angry,0.061258
3,4,Angry,0.036009
4,5,Angry,0.031063
...,...,...,...
233,30,Neutral,0.297270
234,31,Neutral,0.243101
235,32,Neutral,0.330055
236,33,Neutral,0.208880


In [43]:
import plotly.express as px

In [35]:
def get_character_overview(df_video, ID, result):

    '''
    This stacks together:
        1. Portrait
        2. Timeseries for 1 character
        3. Radar_plot for 1 character

        Iteration over characters happens in app.py
    '''
    df_max_rows = df_video.groupby('frame')['probability'].idxmax().reset_index()
    df_max_probs = df_video.loc[df_max_rows['probability']]
    df_radar = plots.get_df_radar(df_video)


    fig_radar = plots.get_radar_plot(df_radar)
    fig_area = plots.get_emotion_landscape(df_video)
    fig_strongest_emotion = plots.get_strongest_emotions_plot(df_max_probs)

    fig = make_subplots(rows=3, cols=2, specs=[[{'type': 'image'}, {'type': 'polar'}],[{'type': 'xy', 'colspan':2},None], [{'type': 'xy', 'colspan':2},None]], horizontal_spacing=0.1, vertical_spacing=0.15)


    # Add trace1, trace2, and the image to their respective subplots
    image_array = result['portraits'][ID]
    fig.add_trace(px.imshow(image_array, ).data[0], row=1, col=1)
    fig.add_trace(fig_radar.data[0], row=1, col=2)

    for i in range(7):
        fig.add_trace(fig_area.data[i], row=2, col=1)

    for i in range(7):
        fig.add_trace(fig_strongest_emotion.data[i], row=3, col=1)
        
    fig.update_layout(
            polar=dict(
                domain=dict(y=[0.5, 1]),
                radialaxis=dict(
                    showticklabels=False,  
                ),
                angularaxis=dict(
                    tickfont=dict(size=14),
                    rotation=90,
                    direction='clockwise'
                )
            ),

            showlegend=True,
            legend=dict(
            x=1.1,
            y=0,
            xanchor='left',
            yanchor='bottom',
            title_font=dict(size=12),
            orientation='v',
            traceorder='reversed'
            ),

            margin=dict(t=80, b=50, l=50, r=50),
            font=dict(size=12),

            # Add the title above the image
            annotations=[
                dict(
                    text=f'Character: {ID}',
                    xref='x domain',
                    yref='y domain',
                    x=0.5,
                    y=1.2,
                    showarrow=False,
                    font=dict(size=16),
                )
            ],
        )


    fig.update_xaxes(showticklabels=False, zeroline=False, visible=False, row=1, col=1)
    fig.update_yaxes(showticklabels=False, zeroline=False, visible=False, row=1, col=1)

    fig.update_xaxes(title_text="Frame Count", title_font=dict(size=12), title_standoff=8, row=2, col=1)
    fig.update_yaxes(title_text="Emotion Probability", title_font=dict(size=12), title_standoff=8, tickmode='linear', dtick=0.2, row=2, col=1)

    fig.update_xaxes(title_text="Emotion Counts", row=1, col=3)
    fig.update_yaxes(title_text="Prevalence", row=1, col=3)


    return fig

In [14]:
df_max_rows = df.groupby('frame')['probability'].idxmax().reset_index()
df_max_probs = df.loc[df_max_rows['probability']]

In [67]:
def get_strongest_emotions_plot(df):
    df_max_rows = df.groupby('frame')['probability'].idxmax().reset_index()
    df_max_probs = df.loc[df_max_rows['probability']]

    fig = px.bar(df_max_probs, x='frame', y='probability', color='emotion', 
                color_discrete_sequence=cb_palette, hover_data={"text": df_max_probs['emotion']})

    # Update the layout
    fig.update_layout(   
        title={
            'text': 'Strongest Emotion per frame',
            'font': {'size': 24, 'color': 'black'},
            'x': 0.5,
            'y': 0.9,
            'yanchor': 'middle'
        },
        xaxis_title='Frame',
        yaxis_title='Probability',
        legend_title='Emotion',
        font=dict(family='Arial', size=14),
        margin=dict(l=50, r=50, t=100, b=50),
        plot_bgcolor='white'
    )

    # Update the legend with customizations
    fig.update_traces(
        hovertemplate='<br>'.join([
            'Emotion: %{fullData.name}',
            'Frame: %{x}',
            'Probability: %{y:.2f}'
        ]),
        hoverlabel=dict(bgcolor='white', font_size=14),
        showlegend=True,
        hoverinfo='all'
    )

    return fig


In [68]:
df = result['new_export']['main_character_data'][1]
df

,frame,emotion,probability
0,1,Angry,0.047856
1,2,Angry,0.062015
2,3,Angry,0.061258
3,4,Angry,0.036009
4,5,Angry,0.031063
...,...,...,...
233,30,Neutral,0.297270
234,31,Neutral,0.243101
235,32,Neutral,0.330055
236,33,Neutral,0.208880


In [70]:
fig = get_strongest_emotions_plot(df)
fig.show()